### try
https://www.benfrederickson.com/matrix-factorization/

In [4]:
import pandas as pd
from scipy.sparse import coo_matrix, linalg
import numpy
from implicit.nearest_neighbours import (
    bm25_weight,
)

In [5]:
# read in triples of user/artist/playcount from the input dataset
data = pd.read_table("usersha1-artmbid-artname-plays.tsv", 
                         usecols=[0, 2, 3], 
                         names=['user', 'artist', 'plays'])

# map each artist and user to a unique numeric value
data['user'] = data['user'].astype("category")
data['artist'] = data['artist'].astype("category")

# create a sparse matrix of all the artist/user/play triples
plays = coo_matrix((data['plays'].astype(float), 
                   (data['artist'].cat.codes,
                    data['user'].cat.codes)))

FileNotFoundError: [Errno 2] No such file or directory: 'usersha1-artmbid-artname-plays.tsv'

In [6]:
artist_factors, _, user_factors = linalg.svds(bm25_weight(plays), 50)

NameError: name 'plays' is not defined

In [8]:
class TopRelated(object):
    def __init__(self, artist_factors):
        # fully normalize artist_factors, so can compare with only the dot product
        norms = numpy.linalg.norm(artist_factors, axis=-1)
        self.factors = artist_factors / norms[:, numpy.newaxis]

    def get_related(self, artistid, N=10):
        scores = self.factors.dot(self.factors[artistid])
        best = numpy.argpartition(scores, -N)[-N:]
        return sorted(zip(best, scores[best]), key=lambda x: -x[1])

In [9]:
tr = TopRelated(artist_factors)
tr.get_related(12, N=5)

NameError: name 'artist_factors' is not defined